# Neural Network - MNIST

## Өгөгдөл унших

In [1]:
import numpy as np
import pickle
import mnist
from matplotlib import pyplot as plt
# % matplotlib inline

training_images, training_labels, test_images, test_labels = mnist.load()


## Neural Network класс

In [2]:



class NN:
    def __init__(self, layer_neuron_counts=None):
        self.layer_neuron_counts = layer_neuron_counts
        self.weights = []
        self.biases = []

        if layer_neuron_counts is not None:
            for i in range(1, len(self.layer_neuron_counts)):
                prev_layer_size = self.layer_neuron_counts[i - 1]
                next_layer_size = self.layer_neuron_counts[i]
                self.weights.append(np.random.randn(prev_layer_size, next_layer_size)*0.01)
                self.biases.append(np.zeros(next_layer_size))

    def train(self, training_data, training_label, epoch, batch_size):
        return

    def relu(self, x, weight, bias):
        h = x @ weight + bias
        h[h <= 0] = 0
        return h

    def predict(self, x):
        for weights, biases in zip(self.weights, self.biases):
            x = self.relu(x, weights, biases)
        return x

    def load(self, filename):
        with open(filename, 'rb') as input:
            nn = pickle.load(input)
            self.weights = nn.weights
            self.biases = nn.biases

    def save(self, filename):
        with open(filename, 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)


In [3]:
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
nn = NN([len(training_images[0]), 16, len(classes)])